In [1]:
#Import der benötigten Libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import librosa
import librosa.display
from IPython.display import Audio
from sklearn.model_selection import train_test_split
import sklearn.preprocessing as skp
import tensorflow as tf
from tensorflow import keras as k
import math

In [2]:
os_dir = os.getcwd()
path_to_soundfilefolder = f'{os_dir}\\Datensatz\\processed_audio'
print(path_to_soundfilefolder)

E:\Notebooks\Genre_Classification\Datensatz\processed_audio


In [ ]:
#Zeug von Librosa
#y_harmonic, y_percussive = librosa.effects.hpss(y)  -------> noch beschreiben, was die einzelnen Attribute aussagen
#tempo, beat_frames = librosa.beat.beat_track(y=y_percussive, sr=sr)
#mfcc = librosa.feature.mfcc(y=y, sr=sr, hop_length=hop_length, n_mfcc=13)
#mfcc_delta = librosa.feature.delta(mfcc)
#beat_mfcc_delta = librosa.util.sync(np.vstack([mfcc, mfcc_delta]), beat_frames)
#chromagram = librosa.feature.chroma_cqt(y=y_harmonic, sr=sr)
#beat_chroma = librosa.util.sync(chromagram, beat_frames, aggregate=np.median)
#beat_features = np.vstack([beat_chroma, beat_mfcc_delta])

In [118]:
#Import der Sound-Dateien\n
import os
import numpy as np

#Zuordnung der Labels zu Zahlen
genre_names = []

#enthält sämtliche Einträge
features = []
#enthält die Ergebnisse sämtlicher Einträge
genre = []

print(path_to_soundfilefolder)

for filename in os.listdir(path_to_soundfilefolder):
    #hier kommen alle Attribute pro Soundfile rein
    attributes = []
    
    #Genre einfügen
    attributes.append(filename.split("_")[0])
    
    #Umwandlung der Soundfiles in repräsentative Zahlenwerte
    print(f"Converting: {filename}")
    y, sr = librosa.load(f'{os_dir}\\Datensatz\\processed_audio\\' + filename)
    
    #extrahieren der aussagekräftigen Eigenschaften
    stft_array = librosa.feature.chroma_stft(y=y, sr=sr)
    stft_mean = np.mean(stft_array)
    stft_var = np.var(stft_array)

    cq_array = librosa.feature.chroma_cqt(y=y, sr=sr)
    cq_mean = np.mean(cq_array)
    cq_var = np.var(cq_array)

    rms_array = librosa.feature.rms(y=y)
    rms_mean = np.mean(rms_array)
    rms_var = np.var(rms_array)

    cent_array = librosa.feature.spectral_centroid(y=y, sr=sr)
    cent_mean = np.mean(cent_array)
    cent_var = np.var(cent_array)

    spec_array = librosa.feature.spectral_bandwidth(y=y, sr=sr)
    spec_mean = np.mean(spec_array)
    spec_var = np.var(spec_array)

    rolloff_array = librosa.feature.spectral_rolloff(y=y, sr=sr)
    rolloff_mean = np.mean(rolloff_array)
    rolloff_var = np.var(rolloff_array)

    zero_crossing_rate_array = librosa.feature.zero_crossing_rate(y)
    zero_crossing_rate_mean = np.mean(zero_crossing_rate_array)
    zero_crossing_rate_var = np.var(zero_crossing_rate_array)

    tonnetz_array = librosa.feature.tonnetz(y=y, sr=sr)
    tonnetz_mean = np.mean(tonnetz_array)
    tonnetz_var = np.var(tonnetz_array)

    y_harmonic_array, y_percussive = librosa.effects.hpss(y)
    tempo1, beat_frames = librosa.beat.beat_track(y=y_percussive, sr=sr)
    tempo = np.float32(tempo1)
    
    y_harmonic_mean = np.mean(y_harmonic_array)
    y_harmonic_var = np.var(y_harmonic_array)

    attributes.extend([stft_mean, stft_var, cq_mean, cq_var, rms_mean, rms_var, cent_mean, cent_var, spec_mean, spec_var, rolloff_mean, rolloff_var, zero_crossing_rate_mean, zero_crossing_rate_var, tonnetz_mean, tonnetz_var, tempo, y_harmonic_mean, y_harmonic_var]) 

    #das mfcc-array besteht aus 20 arrays, für die jeweils mean und var bestimmt werden
    mfcc_array = librosa.feature.mfcc(y=y, sr=sr)
    for i in range(20):
        mfcc_mean = np.mean(mfcc_array[i])
        mfcc_var = np.var(mfcc_array[i])
        attributes.extend([mfcc_mean, mfcc_var])  
        
    
    
    #Anhängen eines Eintrages
    features.append(attributes)
    
    

E:\Notebooks\Genre_Classification\Datensatz\processed_audio
Converting: blues_100_0.wav
Converting: blues_100_1.wav
Converting: blues_100_2.wav
Converting: blues_100_3.wav
Converting: blues_100_4.wav
Converting: blues_100_5.wav
Converting: blues_100_6.wav
Converting: blues_100_7.wav
Converting: blues_100_8.wav
Converting: blues_100_9.wav
Converting: blues_10_0.wav
Converting: blues_10_1.wav
Converting: blues_10_2.wav
Converting: blues_10_3.wav
Converting: blues_10_4.wav
Converting: blues_10_5.wav
Converting: blues_10_6.wav
Converting: blues_10_7.wav
Converting: blues_10_8.wav
Converting: blues_10_9.wav
Converting: blues_11_0.wav
Converting: blues_11_1.wav
Converting: blues_11_2.wav
Converting: blues_11_3.wav
Converting: blues_11_4.wav
Converting: blues_11_5.wav
Converting: blues_11_6.wav
Converting: blues_11_7.wav
Converting: blues_11_8.wav
Converting: blues_11_9.wav
Converting: blues_12_0.wav
Converting: blues_12_1.wav
Converting: blues_12_2.wav
Converting: blues_12_3.wav
Converting: 

Converting: blues_37_2.wav
Converting: blues_37_3.wav
Converting: blues_37_4.wav
Converting: blues_37_5.wav
Converting: blues_37_6.wav
Converting: blues_37_7.wav
Converting: blues_37_8.wav
Converting: blues_37_9.wav
Converting: blues_38_0.wav
Converting: blues_38_1.wav
Converting: blues_38_2.wav
Converting: blues_38_3.wav
Converting: blues_38_4.wav
Converting: blues_38_5.wav
Converting: blues_38_6.wav
Converting: blues_38_7.wav
Converting: blues_38_8.wav
Converting: blues_38_9.wav
Converting: blues_39_0.wav
Converting: blues_39_1.wav
Converting: blues_39_2.wav
Converting: blues_39_3.wav
Converting: blues_39_4.wav
Converting: blues_39_5.wav
Converting: blues_39_6.wav
Converting: blues_39_7.wav
Converting: blues_39_8.wav
Converting: blues_39_9.wav
Converting: blues_3_0.wav
Converting: blues_3_1.wav
Converting: blues_3_2.wav
Converting: blues_3_3.wav
Converting: blues_3_4.wav
Converting: blues_3_5.wav
Converting: blues_3_6.wav
Converting: blues_3_7.wav
Converting: blues_3_8.wav
Converting

Converting: blues_64_7.wav
Converting: blues_64_8.wav
Converting: blues_64_9.wav
Converting: blues_65_0.wav
Converting: blues_65_1.wav
Converting: blues_65_2.wav
Converting: blues_65_3.wav
Converting: blues_65_4.wav
Converting: blues_65_5.wav
Converting: blues_65_6.wav
Converting: blues_65_7.wav
Converting: blues_65_8.wav
Converting: blues_65_9.wav
Converting: blues_66_0.wav
Converting: blues_66_1.wav
Converting: blues_66_2.wav
Converting: blues_66_3.wav
Converting: blues_66_4.wav
Converting: blues_66_5.wav
Converting: blues_66_6.wav
Converting: blues_66_7.wav
Converting: blues_66_8.wav
Converting: blues_66_9.wav
Converting: blues_67_0.wav
Converting: blues_67_1.wav
Converting: blues_67_2.wav
Converting: blues_67_3.wav
Converting: blues_67_4.wav
Converting: blues_67_5.wav
Converting: blues_67_6.wav
Converting: blues_67_7.wav
Converting: blues_67_8.wav
Converting: blues_67_9.wav
Converting: blues_68_0.wav
Converting: blues_68_1.wav
Converting: blues_68_2.wav
Converting: blues_68_3.wav
C

Converting: blues_92_2.wav
Converting: blues_92_3.wav
Converting: blues_92_4.wav
Converting: blues_92_5.wav
Converting: blues_92_6.wav
Converting: blues_92_7.wav
Converting: blues_92_8.wav
Converting: blues_92_9.wav
Converting: blues_93_0.wav
Converting: blues_93_1.wav
Converting: blues_93_2.wav
Converting: blues_93_3.wav
Converting: blues_93_4.wav
Converting: blues_93_5.wav
Converting: blues_93_6.wav
Converting: blues_93_7.wav
Converting: blues_93_8.wav
Converting: blues_93_9.wav
Converting: blues_94_0.wav
Converting: blues_94_1.wav
Converting: blues_94_2.wav
Converting: blues_94_3.wav
Converting: blues_94_4.wav
Converting: blues_94_5.wav
Converting: blues_94_6.wav
Converting: blues_94_7.wav
Converting: blues_94_8.wav
Converting: blues_94_9.wav
Converting: blues_95_0.wav
Converting: blues_95_1.wav
Converting: blues_95_2.wav
Converting: blues_95_3.wav
Converting: blues_95_4.wav
Converting: blues_95_5.wav
Converting: blues_95_6.wav
Converting: blues_95_7.wav
Converting: blues_95_8.wav
C

Converting: classical_119_3.wav
Converting: classical_119_4.wav
Converting: classical_119_5.wav
Converting: classical_119_6.wav
Converting: classical_119_7.wav
Converting: classical_119_8.wav
Converting: classical_119_9.wav
Converting: classical_120_0.wav
Converting: classical_120_1.wav
Converting: classical_120_2.wav
Converting: classical_120_3.wav
Converting: classical_120_4.wav
Converting: classical_120_5.wav
Converting: classical_120_6.wav
Converting: classical_120_7.wav
Converting: classical_120_8.wav
Converting: classical_120_9.wav
Converting: classical_121_0.wav
Converting: classical_121_1.wav
Converting: classical_121_2.wav
Converting: classical_121_3.wav
Converting: classical_121_4.wav
Converting: classical_121_5.wav
Converting: classical_121_6.wav
Converting: classical_121_7.wav
Converting: classical_121_8.wav
Converting: classical_121_9.wav
Converting: classical_122_0.wav
Converting: classical_122_1.wav
Converting: classical_122_2.wav
Converting: classical_122_3.wav
Converti

Converting: classical_145_0.wav
Converting: classical_145_1.wav
Converting: classical_145_2.wav
Converting: classical_145_3.wav
Converting: classical_145_4.wav
Converting: classical_145_5.wav
Converting: classical_145_6.wav
Converting: classical_145_7.wav
Converting: classical_145_8.wav
Converting: classical_145_9.wav
Converting: classical_146_0.wav
Converting: classical_146_1.wav
Converting: classical_146_2.wav
Converting: classical_146_3.wav
Converting: classical_146_4.wav
Converting: classical_146_5.wav
Converting: classical_146_6.wav
Converting: classical_146_7.wav
Converting: classical_146_8.wav
Converting: classical_146_9.wav
Converting: classical_147_0.wav
Converting: classical_147_1.wav
Converting: classical_147_2.wav
Converting: classical_147_3.wav
Converting: classical_147_4.wav
Converting: classical_147_5.wav
Converting: classical_147_6.wav
Converting: classical_147_7.wav
Converting: classical_147_8.wav
Converting: classical_147_9.wav
Converting: classical_148_0.wav
Converti

Converting: classical_170_9.wav
Converting: classical_171_0.wav
Converting: classical_171_1.wav
Converting: classical_171_2.wav
Converting: classical_171_3.wav
Converting: classical_171_4.wav
Converting: classical_171_5.wav
Converting: classical_171_6.wav
Converting: classical_171_7.wav
Converting: classical_171_8.wav
Converting: classical_171_9.wav
Converting: classical_172_0.wav
Converting: classical_172_1.wav
Converting: classical_172_2.wav
Converting: classical_172_3.wav
Converting: classical_172_4.wav
Converting: classical_172_5.wav
Converting: classical_172_6.wav
Converting: classical_172_7.wav
Converting: classical_172_8.wav
Converting: classical_172_9.wav
Converting: classical_173_0.wav
Converting: classical_173_1.wav
Converting: classical_173_2.wav
Converting: classical_173_3.wav
Converting: classical_173_4.wav
Converting: classical_173_5.wav
Converting: classical_173_6.wav
Converting: classical_173_7.wav
Converting: classical_173_8.wav
Converting: classical_173_9.wav
Converti

Converting: classical_196_6.wav
Converting: classical_196_7.wav
Converting: classical_196_8.wav
Converting: classical_196_9.wav
Converting: classical_197_0.wav
Converting: classical_197_1.wav
Converting: classical_197_2.wav
Converting: classical_197_3.wav
Converting: classical_197_4.wav
Converting: classical_197_5.wav
Converting: classical_197_6.wav
Converting: classical_197_7.wav
Converting: classical_197_8.wav
Converting: classical_197_9.wav
Converting: classical_198_0.wav
Converting: classical_198_1.wav
Converting: classical_198_2.wav
Converting: classical_198_3.wav
Converting: classical_198_4.wav
Converting: classical_198_5.wav
Converting: classical_198_6.wav
Converting: classical_198_7.wav
Converting: classical_198_8.wav
Converting: classical_198_9.wav
Converting: classical_199_0.wav
Converting: classical_199_1.wav
Converting: classical_199_2.wav
Converting: classical_199_3.wav
Converting: classical_199_4.wav
Converting: classical_199_5.wav
Converting: classical_199_6.wav
Converti

KeyboardInterrupt: 

In [119]:
#Array an konvertierten Eigenschaften in CSV speichern
arr = features #Ergebnisse aus Funktion davor
headers = ["genre","stft_mean", "stft_var", "cq_mean", "cq_var", "rms_mean", "rms_var", "cent_mean", "cent_var", "spec_mean", "spec_var", "rolloff_mean", "rolloff_var", "zero_crossing_rate_mean", "zero_crossing_rate_var", "tonnetz_mean", "tonnetz_var", "tempo", "y_harmonic_mean", "y_harmonic_var", "mfcc_mean_1", "mfcc_var_1", "mfcc_mean_2", "mfcc_var_2", "mfcc_mean_3", "mfcc_var_3", "mfcc_mean_4", "mfcc_var_4", "mfcc_mean_5", "mfcc_var_5", "mfcc_mean_6", "mfcc_var_6", "mfcc_mean_7", "mfcc_var_7", "mfcc_mean_8", "mfcc_var_8", "mfcc_mean_9", "mfcc_var_9", "mfcc_mean_10", "mfcc_var_10", "mfcc_mean_11", "mfcc_var_11", "mfcc_mean_12", "mfcc_var_12", "mfcc_mean_13", "mfcc_var_13", "mfcc_mean_14", "mfcc_var_14", "mfcc_mean_15", "mfcc_var_15", "mfcc_mean_16", "mfcc_var_16", "mfcc_mean_17", "mfcc_var_17", "mfcc_mean_18", "mfcc_var_18", "mfcc_mean_19", "mfcc_var_19", "mfcc_mean_20", "mfcc_var_20"] # Das sind die Überschriften
df = pd.DataFrame(arr) #Erstelle Datenframe für CSV Output
pd.DataFrame(df).to_csv(f"{os_dir}\Datensatz\\results.csv", header = headers, index=False) #Konvertiere Datenframe in CSV

In [121]:
df = pd.read_csv(f'Datensatz//results.csv')
song_labels = df['genre'] #Erzeuge Genre Datenframe

#Konvertiere String zu Label und erstelle Dictonary dafür
label_index = dict()
for i, x in enumerate(df.genre.unique()):
    label_index[x] = i

labels = []
for x in range(len(song_labels)):
    labels.append(label_index[song_labels[x]])

    
genre_list = pd.DataFrame(labels) #Output Dataframe
features = df.iloc[0:, 1:] # Input Dataframe


In [122]:
print(genre_list)

      0
0     0
1     0
2     0
3     0
4     0
...  ..
1998  2
1999  2
2000  2
2001  2
2002  2

[2003 rows x 1 columns]


In [123]:
print(features)

      stft_mean  stft_var   cq_mean    cq_var  rms_mean   rms_var  \
0      0.430700  0.079948  0.443360  0.068219  0.056834  0.000283   
1      0.401774  0.090328  0.434674  0.064693  0.121461  0.005827   
2      0.346437  0.092592  0.447413  0.059083  0.133687  0.003669   
3      0.336843  0.078507  0.454218  0.071784  0.089527  0.003820   
4      0.360834  0.079291  0.464878  0.071757  0.070490  0.002473   
...         ...       ...       ...       ...       ...       ...   
1998   0.352843  0.094759  0.278276  0.071505  0.262717  0.000468   
1999   0.369021  0.079533  0.258689  0.074967  0.253103  0.000462   
2000   0.398558  0.087204  0.262420  0.077528  0.257514  0.000675   
2001   0.351864  0.083176  0.263956  0.076358  0.256277  0.000341   
2002   0.355300  0.078175  0.367618  0.075574  0.248693  0.000651   

        cent_mean       cent_var    spec_mean       spec_var  ...  \
0     1189.600872   40380.730173  1722.763902   38270.226455  ...   
1     1545.366511  288470.055570 

In [124]:
# normalize
cols = features.columns
min_max_scaler = skp.MinMaxScaler()
np_scaled = min_max_scaler.fit_transform(features)
features_normalized = pd.DataFrame(np_scaled, columns = cols)

In [130]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test = train_test_split(features_normalized,genre_list,test_size = 0.2,random_state = 42)

scaler = skp.StandardScaler()
X_train = pd.DataFrame(scaler.fit_transform(X_train), columns=X_train.columns)
X_test = pd.DataFrame(scaler.transform(X_test), columns=X_train.columns)


In [131]:
#Programmierung des Modells
import tensorflow as tf
from tensorflow import keras

model = keras.Sequential([
    keras.layers.Dense(64, activation=tf.nn.relu),
    keras.layers.Dense(32, activation=tf.nn.relu),
    keras.layers.Dense(16, activation=tf.nn.relu),
    keras.layers.Dense(10, activation=tf.nn.softmax)
])

model.compile(optimizer='adam',
             loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])

model.fit(X_train, Y_train, epochs=300)
model.summary()

Epoch 1/300
51/51 [==============================] - 0s 919us/step - loss: 1.7762 - accuracy: 0.4406
Epoch 2/300
51/51 [==============================] - 0s 1ms/step - loss: 0.2337 - accuracy: 0.9580
Epoch 3/300
51/51 [==============================] - 0s 840us/step - loss: 0.0825 - accuracy: 0.9818
Epoch 4/300
51/51 [==============================] - 0s 1ms/step - loss: 0.0398 - accuracy: 0.9882
Epoch 5/300
51/51 [==============================] - 0s 879us/step - loss: 0.0281 - accuracy: 0.9916
Epoch 6/300
51/51 [==============================] - 0s 959us/step - loss: 0.0185 - accuracy: 0.9939
Epoch 7/300
51/51 [==============================] - 0s 840us/step - loss: 0.0139 - accuracy: 0.9951
Epoch 8/300
51/51 [==============================] - 0s 899us/step - loss: 0.0097 - accuracy: 0.9961
Epoch 9/300
51/51 [==============================] - 0s 820us/step - loss: 0.0075 - accuracy: 0.9972
Epoch 10/300
51/51 [==============================] - 0s 1ms/step - loss: 0.0055 - accuracy: 1.

51/51 [==============================] - 0s 1ms/step - loss: 1.3745e-05 - accuracy: 1.0000
Epoch 80/300
51/51 [==============================] - 0s 1ms/step - loss: 1.6244e-05 - accuracy: 1.0000
Epoch 81/300
51/51 [==============================] - 0s 1ms/step - loss: 1.3639e-05 - accuracy: 1.0000
Epoch 82/300
51/51 [==============================] - 0s 979us/step - loss: 1.1198e-05 - accuracy: 1.0000
Epoch 83/300
51/51 [==============================] - 0s 700us/step - loss: 1.4612e-05 - accuracy: 1.0000
Epoch 84/300
51/51 [==============================] - 0s 780us/step - loss: 1.2385e-05 - accuracy: 1.0000
Epoch 85/300
51/51 [==============================] - 0s 820us/step - loss: 9.1013e-06 - accuracy: 1.0000
Epoch 86/300
51/51 [==============================] - 0s 839us/step - loss: 1.1308e-05 - accuracy: 1.0000
Epoch 87/300
51/51 [==============================] - 0s 820us/step - loss: 1.0769e-05 - accuracy: 1.0000
Epoch 88/300
51/51 [==============================] - 0s 860us/st

51/51 [==============================] - 0s 811us/step - loss: 8.9609e-07 - accuracy: 1.0000
Epoch 157/300
51/51 [==============================] - 0s 899us/step - loss: 8.3747e-07 - accuracy: 1.0000
Epoch 158/300
51/51 [==============================] - 0s 879us/step - loss: 9.1241e-07 - accuracy: 1.0000
Epoch 159/300
51/51 [==============================] - 0s 860us/step - loss: 8.9564e-07 - accuracy: 1.0000
Epoch 160/300
51/51 [==============================] - 0s 1ms/step - loss: 7.1321e-07 - accuracy: 1.0000
Epoch 161/300
51/51 [==============================] - 0s 1ms/step - loss: 9.1890e-07 - accuracy: 1.0000
Epoch 162/300
51/51 [==============================] - 0s 840us/step - loss: 6.0043e-07 - accuracy: 1.0000
Epoch 163/300
51/51 [==============================] - 0s 900us/step - loss: 8.7959e-07 - accuracy: 1.0000
Epoch 164/300
51/51 [==============================] - 0s 860us/step - loss: 7.0295e-07 - accuracy: 1.0000
Epoch 165/300
51/51 [==============================] - 

51/51 [==============================] - 0s 2ms/step - loss: 9.7567e-08 - accuracy: 1.0000
Epoch 233/300
51/51 [==============================] - 0s 2ms/step - loss: 8.5486e-08 - accuracy: 1.0000
Epoch 234/300
51/51 [==============================] - 0s 919us/step - loss: 8.4255e-08 - accuracy: 1.0000
Epoch 235/300
51/51 [==============================] - 0s 839us/step - loss: 8.2485e-08 - accuracy: 1.0000
Epoch 236/300
51/51 [==============================] - ETA: 0s - loss: 5.5879e-08 - accuracy: 1.00 - 0s 939us/step - loss: 9.9861e-08 - accuracy: 1.0000
Epoch 237/300
51/51 [==============================] - 0s 760us/step - loss: 1.0629e-07 - accuracy: 1.0000
Epoch 238/300
51/51 [==============================] - 0s 840us/step - loss: 6.9707e-08 - accuracy: 1.0000
Epoch 239/300
51/51 [==============================] - 0s 859us/step - loss: 6.9815e-08 - accuracy: 1.0000
Epoch 240/300
51/51 [==============================] - 0s 939us/step - loss: 8.3756e-08 - accuracy: 1.0000
Epoch 241

In [ ]:
#bestes Modell aus dem Internet
model1 = k.models.Sequential([
    k.layers.Dense(1024, activation='relu', input_shape=(x_train.shape[1],)),
    k.layers.Dropout(0.3),
    
    k.layers.Dense(512, activation='relu'),
    k.layers.Dropout(0.3),

    k.layers.Dense(256, activation='relu'),
    k.layers.Dropout(0.3),

    k.layers.Dense(128, activation='relu'),
    k.layers.Dropout(0.3),

    k.layers.Dense(64, activation='relu'),
    k.layers.Dropout(0.3),

    k.layers.Dense(10, activation='softmax'),
])
print(model1.summary())

model1.compile(optimizer='rmsprop',
                  loss='sparse_categorical_crossentropy',
                  metrics='accuracy')

model1.fit(x_train, y_train, epochs=300)

In [132]:
import numpy as np
import sklearn.metrics

predictions_confidences = model.predict(X_test)

predictions = []
for entry in predictions_confidences:
        predictions.append(np.argmax(entry))
print(predictions)

[1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 

In [134]:
print(f"{sklearn.metrics.classification_report(Y_test, predictions)}\n")

              precision    recall  f1-score   support

           0       1.00      1.00      1.00       202
           1       0.99      0.99      0.99       199

    accuracy                           1.00       401
   macro avg       1.00      1.00      1.00       401
weighted avg       1.00      1.00      1.00       401




In [ ]:
#test librosa
y, sr = librosa.load(f'{os_dir}\\Datensatz\\processed_audio\\blues_1_0.wav')
print(y, sr)

In [ ]:
tempo, beat_frames = librosa.beat.beat_track(y=y, sr=sr)
print(tempo, beat_frames)

In [ ]:
# Backup Für Standard CSV
df = pd.read_csv(f'Datensatz//features_3_sec.csv')

data = df.iloc[0:, 1:]

song_labels = data['label']
label_index = dict()

for i, x in enumerate(df.label.unique()):
    label_index[x] = i

labels = []

#print(len(song_labels))
for x in range(len(song_labels)):
    labels.append(label_index[song_labels[x]])

genre_list = pd.DataFrame(labels)

# normalize
cols = features.columns
min_max_scaler = skp.MinMaxScaler()
np_scaled = min_max_scaler.fit_transform(features)
features_normalized = pd.DataFrame(np_scaled, columns = cols)
